# GOMC Example for the NVT Ensemble using MoSDeF [1, 2, 5-10, 13-17]


## Import the required packages and specify the force field (FF), box dimensions, density, and mol ratios [1, 2, 5-10, 13-17]. 

In [26]:
import mbuild as mb
import mbuild.formats.charmm_writer as mf_charmm
import mbuild.formats.gomc_conf_writer as gomc_control

FF_to_use = 'trappe-ua'

liquid_box_length_Ang = 45
liquid_box_density_kg_per_m_cubed = 642

pentane_mol_ratio = 0.5
hexane_mol_ratio = 0.5

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Create the pentane and hexane molecules with residue names and minimize their structures [1, 2, 13-17]¶.  

# Generate the lists for the molecules, residues, and mol ratios,  residues/molecules force fields [1, 2, 13-17].  

## Note: For GOMC, the residue names are treated as molecules, so the residue names must be unique for each different molecule [1, 2, 13-17].

In [27]:
pentane = mb.load('../common/pentane.mol2')
pentane.name = 'PEN'
pentane.energy_minimize(forcefield=FF_to_use , steps=10**5)


hexane = mb.load('../common/hexane.mol2')
hexane.name = 'HEX'
hexane.energy_minimize(forcefield=FF_to_use , steps=10**5)


molecule_list = [pentane, hexane]
residues_list = [pentane.name, hexane.name]
mol_ratio_list = [pentane_mol_ratio, hexane_mol_ratio]

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/foyer/forcefield.py:472: UserWarning: Non-atomistic element type detected. Creating custom element for _CH4
  warnings.warn('Non-atomistic element type detected. '
/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/foyer/forcefield.py:472: UserWarning: Non-atomistic element type detected. Creating custom element for _CH3
  warnings.warn('Non-atomistic element type detected. '
/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/foyer/forcefield.py:472: UserWarning: Non-atomistic e

## Build the main liquid simulation box (box 0) for the simulation [1, 2, 13-17]

In [28]:
pentane_hexane_box_liq = mb.fill_box(compound=molecule_list,
                                     density=liquid_box_density_kg_per_m_cubed,
                                     compound_ratio=mol_ratio_list,
                                     box=[liquid_box_length_Ang / 10,
                                          liquid_box_length_Ang / 10,
                                          liquid_box_length_Ang / 10]
                                     )

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/mbuild/formats/xyz.py:63: UserWarning: No matching element found for _CH; the particle will be added to the compound without an element attribute.
  warn(


## Build the Charmm object, which is required to write the FF (.inp), psf, pdb, and GOMC control files [1, 2, 5-10, 13-17]

In [29]:
charmm = mf_charmm.Charmm(pentane_hexane_box_liq,
                          'NVT_pentane_hexane_liq',
                          ff_filename="NVT_pentane_hexane_FF",
                          forcefield_selection=FF_to_use,
                          residues= residues_list,
                          )

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


write_gomcdata: forcefield_selection = trappe-ua, residues = ['PEN', 'HEX']
FF forcefield_selection = {'PEN': 'trappe-ua', 'HEX': 'trappe-ua'}
******************************

GOMC FF writing each residues FF as a group for structure_box_0
forcefield_selection = {'PEN': 'trappe-ua', 'HEX': 'trappe-ua'}
forcefield type from compound = {'PEN': 'trappe-ua', 'HEX': 'trappe-ua'}
coulomb14scale from compound = {'PEN': 0.0, 'HEX': 0.0}
lj14scale from compound = {'PEN': 0.0, 'HEX': 0.0}
all_res_unique_atom_name_dict = {'PEN': ['BD1', 'BD2', 'BD3', 'BD4', 'BD5'], 'HEX': ['BD1', 'BD2', 'BD3', 'BD4', 'BD5', 'BD6']}


/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/mbuild/formats/charmm_writer.py:836: UserWarning: NOTE: All bead names were not found in the Bead to atom naming dictionary (bead_to_atom_name_dict) 
  warn("NOTE: All bead names were not found in the Bead to atom naming dictionary (bead_to_atom_name_dict) ")


## Write the write the FF (.inp), psf, pdb, and GOMC control files [1, 2, 5-10, 13-17]

### Note:  The electrostatics and Ewald are turned off in the GOMC control file (i.e., False) since the n-alkanes beads in the trappe-ua force field have no charge (i.e., the bead charges are all zero)

In [30]:
charmm.write_inp()

charmm.write_psf()

charmm.write_pdb()


gomc_control.write_gomc_control_file(charmm, 'in_NVT.conf',  'NVT', 100, 300,
                                     input_variables_dict={"ElectroStatic": False,
                                                           "Ewald": False,
                                                           }
                                     )

/Users/brad/Programs/anaconda3/envs/GOMC_MOSDEF/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


******************************

The charmm force field file writer (the write_inp function) is running
******************************

The charmm force field file writer (the write_inp function) is running
******************************

writing the GOMC force field file 
No urey bradley terms detected, will use angle_style harmonic
will use CHARMM_torsions  =  K0 + K1 * (1 + Cos[n1*(t) - (d1)] ) + K2 * (1 + Cos[n2*(t) - (d2)] ) + K3 * (1 + Cos[n3*(t) - (d3)] ) + K4 * (1 + Cos[n4*(t) - (d4)] ) + K5 * (1 + Cos[n5*(t) - (d5)] ) 
! RB-torsion to CHARMM dihedral conversion error is OK [error <= 10^(-10)]
! Maximum( |(RB-torsion calc)-(CHARMM dihedral calc)| ) =  1.2878587085651816e-14

NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
self.non_bonded_type = LJ
forcefield_dict = {3: 'CH3_sp3_PEN', 1: 'CH2_sp3_PEN', 2: 'CH3_sp3_HEX', 0: 'CH2_sp3_HEX'}
******************************

The charmm X-plor format psf writer (the write_psf function) is running
write_psf: for

'GOMC_CONTROL_FILE_WRITTEN'